<h2>Objective</h2>

<h3> the main objectuve of this notebook is to make use tranfer learning to make a working protoype model of image classification on the small sample images of plants.</h3>

**evaluation metric**

<u>if the we can make a small dataset from the whole dataset and train any model on basis of it .</u>

then we can use those functions to train various models and find out the best model for our dataset.

In [1]:
# ignoring all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importing the necessary libraries

import numpy as np
import pandas as pd
import cv2

import PIL.Image as Image
import pathlib

import os

import matplotlib.pylab as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [3]:
# importing the metadata.csv with all the path to images and their labels

tdata=pd.read_csv("C:/Users/WELCOME/Documents/PlantProject-master/data/PlantCLEF2022_trusted_training_metadata.csv",delimiter=';')
# wdata=pd.read_csv(r'/home/drowsy/Downloads/data/PlantCLEF2022_web_training_metadata.csv',delimiter=';')
# tdata=pd.read_csv('/home/drowsy/Downloads/data/PlantCLEF2022_trusted_training_metadata.csv',delimiter=';')
# wdata=pd.read_csv('/home/drowsy/Downloads/data/PlantCLEF2022_web_training_metadata.csv',delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/WELCOME/Documents/PlantProject-master/data/PlantCLEF2022_trusted_training_metadata.csv'

**getting all the number of classes for rach type of label**

In [ ]:
print(len(tdata['class'].unique()),
len(tdata['order'].unique()),
len(tdata['family'].unique()),
len(tdata['genus'].unique()),
len(tdata['species'].unique()))

In [ ]:
# directory of the images to train the model.
data_dir='C:/Users/WELCOME/Documents/PlantProject-master/data/images'
# converting the path to pathlib for easy anvigation
data_dir = pathlib.Path(data_dir)
data_dir

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:

# input should be names of columns , 1st should be key and second should be values and 
# 3rd as dataframe with 1 and 2 as columns
def get_labels_on(df,column1,column2):
    df['label']=df[column1].astype('str')+'$'+df[column2].astype('str')
    label_array=df['label'].unique()
    print(label_array)
    label_dict={}
    for label in label_array:
        label_dict.setdefault(str(label).split('$')[0],[]).append(str(label).split('$')[1])
    return label_dict

# output is dict with the target label as key and the classid that is folder name with images in them as values of key
family_labels=get_labels_on(tdata,'family','classid')

make a dict with class names as key and path of image as values
	merge all the keys where classid matches to the same class

In [ ]:
# making a dict with class as key and classid as values


In [ ]:
# making a dict with name of folder as key 
# and putting the path of all images inside the folder as a list in the dict as values.

def get_class_and_path(f,l,path):
    labels=list(path.glob('*/*'))[f:l]
    images_dict={}
#     labels are the list of all the path of subfolders in directory
#     labels=list(dirc.glob('*'))
    for label in labels:
        key=str(label).split('\\')[-2]
        val=str(label).split('\\')[-1]
        images_dict.setdefault(key,[]).append(val)        
#         putting the name of folder inside main dir in key of dict 
#         putting all the images in the folder in list , in value of dict
    return images_dict

In [4]:
plant_images_dict=get_class_and_path(0,5000,data_dir)

NameError: name 'get_class_and_path' is not defined

**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:

def gen_batch(img_dict,label_dict,path):
    X=[]
    y=[]
    cnt=0
    for k_val,i_vals in img_dict.items():
        value = [i for i in label_dict if k_val in label_dict[i]]
        for i_val in i_vals:
            path_val=str(data_dir)+'\\'+k_val+'\\'+i_val
            img=cv2.imread(path_val)
            resized_img=cv2.resize(img,(224,224))
            X.append(resized_img)
            y.append(value[0])
        cnt+=1
        print(k_val,cnt)
    return X,y

In [ ]:
X,y=gen_batch(plant_images_dict,family_labels,data_dir)
len(X)

In [ ]:
# from sklearn.model_selection import train_test_split

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
def img_eng(X,y):
    X=np.array(X)
    y=np.array(y)
    print('converted to array')
    X=X.reshape(X.shape[0],X.shape[1]*X.shape[2]*X.shape[3])
    sc=StandardScaler()
    X=sc.fit_transform(X)
    print('transformed X')
    X=X.reshape(X.shape[0],224,224,3)
    le=LabelEncoder()
    y=le.fit_transform(y)
    print('transformed y')
    y=to_categorical(y,num_classes=483,dtype='int')
    return X,y

In [ ]:
X,y=img_eng(X,y)
X.shape

converted to array



**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [37]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(483, activation='relu'))
model.add(Dense(483, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [38]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 26, 26, 128)     

In [ ]:
model.fit(X,y,epochs=20,verbose=1)1

In [74]:
model.predict(X)[9].argmax()

1/1 [==============================] - 0s 226ms/step


7

In [60]:
X

array([[[[-6.03801420e-01,  1.14673384e+00,  2.43102629e-01],
         [-1.09774063e+00,  1.75811631e-01, -8.72858931e-01],
         [-1.05321270e+00,  2.41410785e-01, -1.01499921e+00],
         ...,
         [-9.10296653e-01, -3.73009687e-01, -1.21037671e+00],
         [-7.45385638e-01, -5.95412382e-02, -1.09713176e+00],
         [-8.12736189e-01,  2.34575529e-02, -1.15378628e+00]],

        [[-6.98627428e-01,  6.73164285e-01, -1.69566320e-01],
         [-1.07101448e+00,  1.55016840e-01, -1.01201522e+00],
         [-1.01026717e+00,  8.96524359e-02, -1.09399988e+00],
         ...,
         [-1.02078773e+00, -8.35362571e-01, -1.41085785e+00],
         [-8.62665068e-01, -6.12665793e-01, -1.17138594e+00],
         [-7.76945907e-01, -5.20666457e-01, -1.17386669e+00]],

        [[-7.26601893e-01,  6.28125292e-01, -3.79249241e-01],
         [-1.14071871e+00,  3.71699871e-01, -1.06201888e+00],
         [-9.95776205e-01,  1.87085054e-01, -1.13167582e+00],
         ...,
         [-1.09096028e+0

In [50]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the ResNet-50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add a global average pooling layer and a dense output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(80000, activation='softmax')(x)

# Create the final model by combining the ResNet-50 base model with the new dense layers
model1 = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the base ResNet-50 layers to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with the Adam optimizer and categorical crossentropy loss
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


**-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [ ]:
# exporting the model

In [12]:
from joblib import load,dump

dump(model,filename='./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib')

['./models/model_pre_efficientnet_v2_imagenet21k_ft1k_xl.joblib']

In [ ]:
loaded_model=load(filename='model_pre_name.joblib')